
### Kerry Back, Rice University, Original Source

<a target="_blank" href="https://colab.research.google.com/github/kerryback/mgmt638/blob/main/notebooks/11-backtest_sector.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


### MBA candidate Ray Lee add extension based on professor Kerry's handout

In [1]:
from google.colab import drive
import sys
from joblib import load
import pandas as pd

drive.mount('/content/drive', force_remount=True)

folder_path = '/content/drive/MyDrive/MGMT767'
sys.path.append(folder_path)

Mounted at /content/drive


In [3]:
df = pd.read_csv(folder_path + "/merged_database_04302024.csv")
df.tail()

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,closeunadj,roe,accruals,agr,sector,mktvol,ShortVolume,ShortExemptVolume,TotalVolume,relss
1122503,3158751,ZYXI,2024-03-29,410.8,8.9,-0.031323,0.174704,106661.40,0.066006,12.77,12.77,0.168182,-0.138665,0.038452,Healthcare,0.115083,15714.25,0.0,30400.75,0.516903
1122504,3158752,ZYXI,2024-04-05,397.9,8.6,-0.000808,0.025000,114093.75,0.065290,12.37,12.37,0.168182,-0.138665,0.038452,Healthcare,0.095117,21565.40,48.2,38904.40,0.554318
1122505,3158753,ZYXI,2024-04-12,397.6,8.6,-0.036408,0.057566,131059.40,0.065401,12.36,12.36,0.168182,-0.138665,0.038452,Healthcare,0.102599,15484.80,519.8,30408.40,0.509228
1122506,3158754,ZYXI,2024-04-19,383.1,8.3,-0.044500,0.051089,117266.80,0.065897,11.91,11.91,0.168182,-0.138665,0.038452,Healthcare,0.109365,26955.00,1076.8,44156.00,0.610449
1122507,3158755,ZYXI,2024-04-26,366.1,7.9,-0.014938,0.067726,133455.20,0.064197,11.38,11.38,0.168182,-0.138665,0.038452,Healthcare,0.106969,28044.40,735.4,40218.60,0.697299


In [4]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(max_depth=4, n_estimators=200)


df["target"] = df.groupby("date", group_keys=False).ret.apply(
    lambda x: 100 * (x-x.median())
)

"""
# could use this instead

df["target"] = df.groupby("date", group_keys=False).ret.apply(
    lambda x: 100 * x.rank(pct=True)
)
"""

'\n# could use this instead\n\ndf["target"] = df.groupby("date", group_keys=False).ret.apply(\n    lambda x: 100 * x.rank(pct=True)\n)\n'

In [5]:
df_today = df.dropna()
df_today = df_today[df_today.date==df_today.date.max()].copy()

In [6]:
df_today.head()

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,roe,accruals,agr,sector,mktvol,ShortVolume,ShortExemptVolume,TotalVolume,relss,target
299,747,A,2024-04-26,38897.2,6.3,0.037746,0.070587,1634307.6,0.041640,132.73,...,0.211535,-0.124080,0.000672,Healthcare,0.106969,273074.4,326.4,383981.0,0.711166,2.488820
599,1138,AA,2024-04-26,6379.7,1.5,0.037996,-0.166202,9659967.6,0.080297,35.53,...,-0.142357,-0.162257,-0.010513,Basic Materials,0.106969,1016169.4,4941.8,2453575.0,0.414159,2.513846
801,2067,AADI,2024-04-26,45.2,0.4,-0.048913,-0.726474,153926.0,0.124697,1.84,...,-0.525284,-0.048738,-0.080235,Healthcare,0.106969,28270.2,408.0,55727.4,0.507294,-6.177065
1282,3925,AAME,2024-04-26,40.0,0.4,0.000000,0.207804,6528.0,0.082149,1.96,...,-0.001653,-0.026993,0.014440,Financial Services,0.106969,908.6,20.0,2444.8,0.371646,-1.285760
1461,4104,AAN,2024-04-26,227.1,0.3,-0.054813,-0.252845,208955.0,0.101906,7.48,...,0.004031,-0.253584,-0.004372,Industrials,0.106969,19887.4,808.4,50973.8,0.390149,-6.767044


# Prediction Random Forest

In [52]:
#df_rf = df_today
df_rf = df_Mar
df_rf = df_rf[df_rf.closeunadj>5]
df_rf = df_rf.dropna()
df_rf["rnk"] = df_rf.groupby("date", group_keys=False).marketcap.rank(
    ascending=False,
    method="first"
)
#df_rf = df_rf[(df_rf.rnk>1000) & (df_rf.rnk<=3000)]
df_rf = df_rf.sort_values(by=["date", "ticker"])

In [53]:
df_rf.tail()

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals,agr,sector,mktvol,ShortVolume,ShortExemptVolume,TotalVolume,relss,target,rnk
1121139,3155557,ZUO,2024-03-29,1259.3,10.0,0.036364,-0.120301,1084495.4,0.054717,8.80,...,-0.415629,0.055806,Technology,0.115083,149793.50,53.75,288678.50,0.518894,1.787529,1758.0
1121380,3156597,ZVRA,2024-03-29,208.6,3.1,0.006944,0.212066,194799.8,0.089036,5.76,...,-0.246647,0.043451,Healthcare,0.115083,52472.00,522.50,97915.50,0.535891,-1.154390,2599.0
1121597,3157323,ZWS,2024-03-29,5601.9,3.5,0.027633,0.607088,662163.4,0.034990,32.57,...,-0.088216,-0.017451,Industrials,0.115083,97901.50,392.25,220366.50,0.444267,0.914445,886.0
1121959,3157751,ZYME,2024-03-29,742.4,1.6,0.000000,0.456098,463455.0,0.062049,10.52,...,-0.000819,-0.028176,Healthcare,0.115083,47507.75,139.25,83128.25,0.571499,-1.848834,2028.0
1122503,3158751,ZYXI,2024-03-29,410.8,8.9,-0.031323,0.174704,106661.4,0.066006,12.77,...,-0.138665,0.038452,Healthcare,0.115083,15714.25,0.00,30400.75,0.516903,-4.981176,2338.0


In [54]:
df_rf.columns

Index(['index', 'ticker', 'date', 'marketcap', 'pb', 'ret', 'mom', 'volume',
       'volatility', 'closeadj', 'closeunadj', 'roe', 'accruals', 'agr',
       'sector', 'mktvol', 'ShortVolume', 'ShortExemptVolume', 'TotalVolume',
       'relss', 'target', 'rnk'],
      dtype='object')

In [55]:
from urllib.request import urlopen
file_path_MODEL = folder_path + "/RF_relss.joblib"
forest = load(file_path_MODEL)

In [56]:
features = [
    "marketcap",
    "pb",
    "mom",
    "volume",
    "volatility",
    "roe",
    "accruals",
    "agr",
    "relss"
]
features.sort()

for x in features:
    df_rf[x+"_vol"] = df_rf[x]*df_rf.mktvol
features_final = features + [x+"_vol" for x in features]
df_rf["predict"] = forest.predict(X=df_rf[features_final])

In [57]:
df_rf.sort_values(by='marketcap', ascending=False).head(15)

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals_vol,agr_vol,marketcap_vol,mom_vol,pb_vol,relss_vol,roe_vol,volatility_vol,volume_vol,predict
667772,1881735,MSFT,2024-03-29,3185725.2,13.4,-0.018706,0.474479,19170696.80,0.024210,428.740,...,-0.010763,0.007613,366621.458947,0.054604,1.542107,0.040890,0.044187,0.002786,2.206213e+06,0.011759
667771,1881734,MSFT,2024-03-22,3094182.3,13.0,0.029586,0.457832,26450626.20,0.025982,416.420,...,-0.010763,0.007613,356086.466301,0.052688,1.496073,0.046735,0.044187,0.002990,3.044006e+06,0.018327
667769,1881732,MSFT,2024-03-08,3087346.3,13.0,-0.022335,0.620994,18631647.20,0.025412,415.500,...,-0.011283,0.007980,372438.138550,0.074913,1.568239,0.040412,0.046319,0.003065,2.247605e+06,0.005686
667768,1881731,MSFT,2024-03-01,3049005.2,12.8,0.012575,0.631036,20938940.25,0.025455,410.340,...,-0.012736,0.009009,415194.337937,0.085930,1.743023,0.048703,0.052285,0.003466,2.851333e+06,0.011759
667770,1881733,MSFT,2024-03-15,3018391.8,12.7,0.025110,0.702448,20319863.20,0.026108,406.220,...,-0.011419,0.008077,368529.123918,0.085765,1.550601,0.045345,0.046880,0.003188,2.480944e+06,0.016166
2653,7021,AAPL,2024-03-01,2818452.1,38.0,-0.015670,0.316860,46020627.00,0.032295,182.520,...,-0.032680,0.000674,383799.067862,0.043148,5.174601,0.054715,0.212491,0.004398,6.266799e+06,0.018542
2654,7022,AAPL,2024-03-08,2791120.0,37.7,-0.049705,0.235516,70135534.80,0.030082,179.660,...,-0.028951,0.000597,336703.251356,0.028411,4.547892,0.039015,0.188242,0.003629,8.460712e+06,0.008061
2656,7024,AAPL,2024-03-22,2665577.5,36.0,-0.001970,0.182441,72398287.20,0.029267,172.620,...,-0.027619,0.000570,306761.522302,0.020996,4.142973,0.040841,0.179580,0.003368,8.331781e+06,0.014134
2657,7025,AAPL,2024-03-29,2660327.3,35.9,-0.004644,0.145015,71599265.80,0.029269,172.280,...,-0.027619,0.000570,306157.315767,0.016689,4.131464,0.039367,0.179580,0.003368,8.239828e+06,0.008061
2655,7023,AAPL,2024-03-15,2636392.3,35.6,0.011070,0.278467,77224293.40,0.029377,170.730,...,-0.029301,0.000605,321889.075044,0.033999,4.346565,0.045660,0.190521,0.003587,9.428664e+06,0.018542


In [66]:
df_rf = df_rf.sort_values(by='predict', ascending=False)

In [67]:
df_rf.head(40)

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals_vol,agr_vol,marketcap_vol,mom_vol,pb_vol,relss_vol,roe_vol,volatility_vol,volume_vol,predict
931621,2612978,SOUN,2024-03-08,1865.5,66.2,-0.028998,-0.460317,1.812580e+08,0.200763,6.035,...,-0.092363,0.015935,225.042247,-0.055530,7.985954,0.064638,-0.438116,0.024219,2.186583e+07,22.671293
906176,2543515,SIDU,2024-03-08,5.4,0.6,-0.327922,-0.950057,8.011603e+06,0.347001,6.160,...,-0.046338,0.014164,0.651422,-0.114609,0.072380,0.065017,-0.239677,0.041860,9.664696e+05,3.228922
310994,870979,DTSS,2024-03-15,20.2,7.8,-0.002516,-0.873156,3.332096e+06,0.668957,7.950,...,0.091042,-0.025669,2.466309,-0.106608,0.952337,0.061566,-0.243181,0.081676,4.068307e+05,2.931716
787559,2210602,PHUN,2024-03-08,19.2,2.9,0.480226,-0.637703,1.336313e+06,0.489013,7.080,...,-0.141120,-0.023411,2.316168,-0.076928,0.349838,0.068341,-0.275386,0.058991,1.612044e+05,2.767647
642904,1815452,MINM,2024-03-22,16.8,123.5,-0.045526,0.270936,1.632583e+06,0.684202,6.370,...,-0.206341,-0.026243,1.933387,0.031180,14.212698,0.056750,-0.219478,0.078740,1.878819e+05,2.714166
1065704,2989256,VRM,2024-03-22,23.3,0.2,0.003846,-0.857557,1.235120e+06,0.150645,13.000,...,0.069054,-0.002539,2.681424,-0.098690,0.023017,0.062186,-0.150120,0.017337,1.421408e+05,2.691467
1065701,2989253,VRM,2024-03-01,17.4,0.1,0.203015,-0.754696,1.982680e+05,0.152827,9.950,...,0.115348,-0.003312,2.369422,-0.102770,0.013617,0.077498,-0.073706,0.020811,2.699889e+04,2.627009
1065702,2989254,VRM,2024-03-08,20.9,0.1,0.047619,-0.775258,1.358926e+05,0.155648,11.970,...,0.102185,-0.002934,2.521245,-0.093522,0.012063,0.066550,-0.065295,0.018776,1.639323e+04,2.609610
1065703,2989255,VRM,2024-03-15,21.9,0.1,0.036683,-0.762923,8.027040e+04,0.155988,12.540,...,0.073261,-0.002694,2.673870,-0.093149,0.012209,0.068054,-0.159267,0.019045,9.800577e+03,2.553139
221355,614882,CMAX,2024-03-01,28.7,0.1,-0.001300,-0.900000,6.691550e+04,0.227177,7.690,...,-0.027441,0.011572,3.908185,-0.122556,0.013617,0.103149,-0.048878,0.030936,9.112132e+03,2.553139


In [60]:
df_rf.head(40).to_csv(folder_path + "/RF_DaystoCover_long.csv", index=False)

In [61]:
df_rf_long = df_rf.head(40)
df_rf_long['sector'].value_counts()

sector
Healthcare                19
Technology                 9
Consumer Cyclical          5
Industrials                2
Real Estate                2
Communication Services     1
Consumer Defensive         1
Basic Materials            1
Name: count, dtype: int64

In [62]:
df_rf.tail(40)

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals_vol,agr_vol,marketcap_vol,mom_vol,pb_vol,relss_vol,roe_vol,volatility_vol,volume_vol,predict
742183,2090001,ONCT,2024-03-08,29.6,0.8,-0.043868,-0.474110,12697.60,0.184835,10.030,...,-0.021429,-0.017433,3.570759,-0.057194,0.096507,0.043705,-0.115651,0.022297,1531.759009,-1.554797
742184,2090002,ONCT,2024-03-15,28.4,0.9,-0.163712,-0.341245,11192.40,0.185491,9.590,...,-0.021688,-0.017644,3.467485,-0.041664,0.109885,0.027773,-0.117051,0.022647,1366.530802,-1.561740
623306,1752640,MBRX,2024-03-29,16.4,0.5,0.035933,-0.465545,43022.16,0.150594,5.705,...,-0.006159,-0.008341,1.887354,-0.053576,0.057541,0.048991,-0.092536,0.017331,4951.100951,-1.571528
742182,2090000,ONCT,2024-03-01,26.7,0.7,0.109513,-0.499112,7363.50,0.185795,9.040,...,-0.019400,-0.017687,3.635838,-0.067966,0.095322,0.054764,-0.116143,0.025300,1002.715085,-1.572127
551277,1552990,KALA,2024-03-01,19.0,1.7,0.038244,-0.380870,56403.75,0.096790,7.060,...,-0.095304,-0.029217,2.587300,-0.051864,0.231495,0.057560,-0.355334,0.013180,7680.707674,-1.578408
551281,1552994,KALA,2024-03-29,21.5,1.9,-0.002503,-0.546564,17432.60,0.093941,7.990,...,-0.110084,-0.012393,2.474275,-0.062900,0.218657,0.045511,-0.325494,0.010811,2006.188495,-1.581366
151215,411765,BNTC,2024-03-22,13.2,0.9,-0.029412,-0.269853,7173.60,0.127566,5.100,...,-0.040369,0.021055,1.519090,-0.031055,0.103574,0.035136,-0.235163,0.014681,825.556359,-1.582535
642654,1814152,MIND,2024-03-08,8.5,0.4,0.039669,0.087324,1614.40,0.067396,6.050,...,0.031471,-0.003947,1.025387,0.010534,0.048253,0.037672,-0.002723,0.008130,194.751114,-1.588720
551280,1552993,KALA,2024-03-22,20.3,1.8,0.061089,-0.488456,11344.40,0.093019,7.530,...,-0.080543,-0.024692,2.336176,-0.056213,0.207149,0.049311,-0.300298,0.010705,1305.542763,-1.588795
551278,1552991,KALA,2024-03-08,19.7,1.8,0.023192,-0.584478,24255.80,0.097450,7.330,...,-0.084428,-0.025883,2.376485,-0.070508,0.217141,0.028301,-0.314784,0.011756,2926.067931,-1.588795


In [63]:
df_rf.tail(40).to_csv(folder_path + "/RF_DaystoCover_short.csv", index=False)

In [64]:
df_rf_short = df_rf.tail(40)
df_rf_short['sector'].value_counts()

sector
Healthcare                33
Technology                 5
Consumer Defensive         1
Communication Services     1
Name: count, dtype: int64

In [65]:
df_rf.to_csv(folder_path + '/portfoio_0408.csv', index=False)

#Prediction Neural Network

In [33]:
df_nn = df_today
df_nn = df_nn[df_nn.closeunadj>5]
df_nn = df_nn.dropna()
df_nn["rnk"] = df_nn.groupby("date", group_keys=False).marketcap.rank(
    ascending=False,
    method="first"
)
#df_nn = df_nn[(df_nn.rnk>1000) & (df_nn.rnk<=3000)]
df_nn = df_nn.sort_values(by=["date", "ticker"])

In [34]:
import torch.nn as nn

In [35]:
class NeuralModel(nn.Module):
    def __init__(self, input_size):
        super(NeuralModel, self).__init__()
        self.batch_norm = nn.BatchNorm1d(input_size)
        self.hidden_layer_1 = nn.Linear(input_size, 1024)
        self.dropout_1 = nn.Dropout(0.25)
        self.hidden_layer_2 = nn.Linear(1024, 1024)
        self.dropout_2 = nn.Dropout(0.25)
        self.hidden_layer_3 = nn.Linear(1024, 1024)
        self.dropout_3 = nn.Dropout(0.25)
        self.output_layer = nn.Linear(1024, 1)
        #self.Sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.batch_norm(x)
        x = torch.relu(self.hidden_layer_1(x))
        x = self.dropout_1(x)
        x = torch.relu(self.hidden_layer_2(x))
        x = self.dropout_2(x)
        x = torch.relu(self.hidden_layer_3(x))
        x = self.dropout_3(x)
        #x = self.Sigmoid(self.output_layer(x))
        x = self.output_layer(x)

        return x


In [36]:
features = [
    "marketcap",
    "pb",
    "mom",
    "volume",
    "volatility",
    "roe",
    "accruals",
    "agr",
    "relss"
]
features.sort()

for x in features:
    df_nn[x+"_vol"] = df_nn[x]*df_nn.mktvol
features_final = features + [x+"_vol" for x in features]

In [37]:
import torch

# Create an instance of your model
neural_model = NeuralModel(len(features_final))  # Replace your_input_size with the appropriate value

# Specify the folder path where you want to save the model
folder_path = '/content/drive/MyDrive/MGMT767'

# Save the model state dictionary
neural_model.load_state_dict(torch.load(folder_path + '/mnist_1_Short_Volumn_0412.pth'))

<All keys matched successfully>

In [38]:
import torch

# Assuming df_long is a pandas DataFrame and features_final is a list of feature names
# Convert your DataFrame's selected features to a PyTorch tensor and make predictions
predictions = neural_model(torch.tensor(df_nn[features_final].values, dtype=torch.float32)).detach().numpy()

# Assign predictions to a new column in your DataFrame
df_nn["predict"] = predictions


In [39]:
df_nn.sort_values(by='marketcap', ascending=False).head(15)

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals_vol,agr_vol,marketcap_vol,mom_vol,pb_vol,relss_vol,roe_vol,volatility_vol,volume_vol,predict
667776,1881739,MSFT,2024-04-26,2965635.7,12.4,0.018040,0.512615,20481581.0,0.027428,399.12,...,-0.011155,0.006523,317230.630313,0.054834,1.326414,0.039944,0.040153,0.002934,2.190891e+06,-0.601398
2661,7029,AAPL,2024-04-26,2547910.4,34.4,0.026061,0.049541,60905152.0,0.031340,165.00,...,-0.025671,0.000530,272547.036769,0.005299,3.679728,0.049453,0.166919,0.003352,6.514954e+06,0.032376
442235,1246134,GOOGL,2024-04-26,1915801.0,6.8,0.115906,0.430320,24053096.4,0.043193,154.09,...,-0.009457,0.002610,204931.023316,0.046031,0.727388,0.029311,0.031579,0.004620,2.572932e+06,-0.270566
718251,2022807,NVDA,2024-04-26,1905000.0,44.3,0.151378,2.477990,52229317.0,0.056941,762.00,...,0.005568,0.013867,203775.652804,0.265068,4.738720,0.049144,0.099273,0.006091,5.586910e+06,-0.077325
62383,165480,AMZN,2024-04-26,1813948.4,9.0,0.028575,0.672307,39129859.0,0.032552,174.63,...,-0.032951,0.003686,194035.968169,0.071916,0.962720,0.031668,0.018388,0.003482,4.185676e+06,0.334192
635297,1790355,META,2024-04-26,1226442.3,8.0,-0.078533,1.396186,14978689.6,0.053874,481.07,...,-0.022593,0.004901,131191.118272,0.149348,0.855751,0.057592,0.033779,0.005763,1.602253e+06,0.660711
593565,1670674,LLY,2024-04-26,690113.9,64.1,0.009913,1.016929,2606067.6,0.037217,726.31,...,0.009673,0.007210,73820.687917,0.108780,6.856703,0.039232,0.050676,0.003981,2.787680e+05,-0.202607
105335,284316,AVGO,2024-04-26,558288.2,7.9,0.115679,1.180930,2709965.4,0.063649,1204.71,...,-0.022362,0.038812,59719.444833,0.126323,0.845054,0.052249,0.035975,0.006808,2.898819e+05,-0.242700
1045602,2939913,V,2024-04-26,544503.5,13.7,0.017570,0.219771,8028942.4,0.017835,269.78,...,-0.006735,0.001687,58244.911373,0.023509,1.465473,0.053808,0.049823,0.001908,8.588467e+05,0.524148
548442,1545484,JPM,2024-04-26,535173.0,1.6,0.041389,0.427744,12498020.4,0.026680,185.80,...,0.011805,0.001478,57246.838550,0.045755,0.171150,0.037585,0.016208,0.002854,1.336899e+06,-0.143361


In [40]:
df_nn = df_nn.sort_values(by='predict', ascending=False)

In [41]:
df_nn.head(40)

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals_vol,agr_vol,marketcap_vol,mom_vol,pb_vol,relss_vol,roe_vol,volatility_vol,volume_vol,predict
220017,611123,CLSK,2024-04-26,3406.7,4.2,0.118023,4.043928,34242370.2,0.214591,17.200,...,-0.001018,0.016530,364.410770,0.432574,0.449269,0.062085,-0.013618,0.022955,3.662867e+06,6.513800
955664,2682086,STTK,2024-04-26,454.8,3.1,0.105428,2.218430,350683.4,0.355311,9.580,...,-0.004691,-0.007320,48.649431,0.237303,0.331603,0.059853,-0.067472,0.038007,3.751220e+04,4.440931
945164,2651354,SSNT,2024-04-26,70.0,9.4,0.058466,3.289224,102786.2,0.314037,13.170,...,-0.020445,-0.001236,7.487819,0.351844,1.005507,0.067947,-0.013228,0.033592,1.099492e+04,4.275617
865345,2426719,ROOT,2024-04-26,751.0,4.5,0.251944,13.036554,494955.8,0.324982,51.440,...,-0.060797,0.000752,80.333604,1.394505,0.481360,0.057799,-0.078747,0.034763,5.294485e+04,3.879533
699245,1969908,NKTX,2024-04-26,360.2,1.3,-0.057613,0.870021,914308.0,0.208874,7.290,...,-0.010861,-0.005697,38.530179,0.093065,0.139060,0.059239,-0.040718,0.022343,9.780247e+04,3.580482
285012,797076,DAVE,2024-04-26,407.7,4.7,0.312519,6.171123,62608.0,0.172580,33.310,...,-0.100450,-0.002414,43.611199,0.660118,0.502754,0.059522,-0.059237,0.018461,6.697106e+03,3.097499
520496,1465241,INO,2024-04-26,226.2,1.9,0.091942,0.123900,492118.0,0.167544,9.680,...,-0.007371,-0.018243,24.196353,0.013253,0.203241,0.064659,-0.092636,0.017922,5.264129e+04,3.013205
966930,2715650,SYRE,2024-04-26,1171.7,6.4,0.035483,8.682726,360115.2,0.198710,32.410,...,-0.046903,0.078149,125.335398,0.928781,0.684601,0.073181,-0.151316,0.021256,3.852111e+04,2.360313
896487,2514484,SEZL,2024-04-26,314.8,14.2,0.055655,1.520000,66497.2,0.277038,55.880,...,0.200394,0.006482,33.673793,0.162593,1.518958,0.065956,0.043381,0.029634,7.113129e+03,2.167319
934025,2620600,SPIR,2024-04-26,235.9,3.3,-0.008016,1.136009,373695.6,0.129953,9.980,...,-0.051456,-0.000992,25.233951,0.121518,0.352997,0.062751,-0.081591,0.013901,3.997379e+04,2.162747


In [42]:
df_nn.head(40).to_csv(folder_path + "/NN_DaystoCover_long.csv", index=False)

In [43]:
df_nn_long = df_nn.head(40)
df_nn_long['sector'].value_counts()

sector
Healthcare                15
Industrials                7
Financial Services         5
Consumer Cyclical          5
Technology                 4
Basic Materials            3
Communication Services     1
Name: count, dtype: int64

In [44]:
df_nn.tail(40)

,index,ticker,date,marketcap,pb,ret,mom,volume,volatility,closeadj,...,accruals_vol,agr_vol,marketcap_vol,mom_vol,pb_vol,relss_vol,roe_vol,volatility_vol,volume_vol,predict
172016,468368,CABA,2024-04-26,643.5,2.7,-0.174663,1.019790,1025091.2,0.113523,13.340,...,-0.009118,0.025281,68.834453,0.109086,0.288816,0.026148,-0.043681,0.012143,109652.823340,-1.205412
501595,1413983,IBTA,2024-04-26,2975.2,106.5,0.103776,2.988180,2568075.5,0.199160,98.000,...,-0.017433,0.001445,318.253712,0.319642,11.392182,0.047817,-0.073437,0.021304,274704.074258,-1.225376
768947,2159461,PBT,2024-04-26,566.3,2562.4,0.004115,-0.487639,127110.6,0.056212,12.150,...,-0.010202,-0.001154,60.576458,-0.052162,274.096973,0.047268,-0.004284,0.006013,13596.874275,-1.232602
692577,1951665,NEXT,2024-04-26,1568.5,5.4,0.047463,-0.100813,969932.2,0.060454,6.110,...,-0.025669,0.083233,167.780636,-0.010784,0.577632,0.057073,-0.058739,0.006467,103752.528730,-1.243834
738175,2078779,OLMA,2024-04-26,555.2,2.2,0.002014,1.706024,603372.2,0.130110,9.930,...,-0.006391,0.007436,59.389104,0.182491,0.235331,0.034422,-0.047778,0.013918,64542.028314,-1.251772
89043,240784,ASMB,2024-04-26,73.5,1.8,-0.056674,0.109402,15888.2,0.075243,13.410,...,-0.172659,0.012362,7.862210,0.011703,0.192544,0.035140,-0.125895,0.008049,1699.542429,-1.255062
1050887,2954355,VERA,2024-04-26,2069.2,20.3,0.043421,5.278714,1112655.4,0.231024,38.000,...,-0.003112,0.006629,221.339937,0.564658,2.171468,0.044018,-0.078438,0.024712,119019.464819,-1.255828
833931,2345727,RANI,2024-04-26,354.6,27.6,-0.055163,-0.325052,555574.2,0.143052,7.070,...,0.086674,-0.014956,37.931153,-0.034770,2.952340,0.046346,-0.170528,0.015302,59429.131384,-1.289106
910452,2558113,SKIL,2024-04-26,43.2,0.2,0.357678,-0.678261,146839.6,0.096108,5.340,...,-1.586810,0.000053,4.621054,-0.072553,0.021394,0.048613,-1.586570,0.010281,15707.262650,-1.295600
913867,2565828,SLDB,2024-04-26,328.5,2.6,0.032184,1.829218,155373.0,0.208175,8.700,...,-0.001270,-0.011595,35.139266,0.195669,0.278119,0.045157,-0.066438,0.022268,16620.070605,-1.298410


In [45]:
df_nn.tail(40).to_csv(folder_path + "/NN_DaystoCover_short.csv", index=False)

In [46]:
df_nn_short = df_nn.tail(40)
df_nn_short['sector'].value_counts()

sector
Healthcare                26
Technology                 4
Energy                     3
Communication Services     2
Consumer Defensive         2
Financial Services         2
Real Estate                1
Name: count, dtype: int64

In [47]:
#df_nn.to_csv(folder_path + '/nn_portfolio_0415.csv', index=False)